In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
import matplotlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import SVG
import keras.backend as K
from keras.layers import Input, Dense, LSTM, GRU, Embedding, Lambda, Concatenate, Add
from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.utils.vis_utils import model_to_dot
from tqdm import tqdm,trange
from sklearn.utils import shuffle
import time as time
from keras.utils import plot_model

Using TensorFlow backend.
